# 01. Setup y Carga de Datos M5

---

**Fase 1** - Setup y Exploración de Datos

**Objetivo:** Cargar y explorar los datos M5 de Kaggle, comprender su estructura y preparar el dataset base para análisis de forecasting de urgencias.

**Input:** 
- `data/raw/sales_train_evaluation.csv` (ventas diarias por producto)
- `data/raw/calendar.csv` (mapeo de días a fechas)
- `data/raw/sell_prices.csv` (precios por tienda y fecha)

**Output:**
- `data/processed/sales_daily.csv` (ventas en formato largo)
- `data/processed/sales_weekly.csv` (ventas agregadas semanalmente)

---

## 1. Configuración Inicial

In [ ]:
# Imports estándar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from datetime import datetime

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
np.random.seed(42)

# Estilo de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('viridis')

print("✓ Librerías cargadas correctamente")
print(f"  - Pandas version: {pd.__version__}")
print(f"  - NumPy version: {np.__version__}")

In [ ]:
# Configuración de paths
PROJECT_ROOT = Path.cwd().parent
DATA_RAW = PROJECT_ROOT / 'data' / 'raw'
DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed'
FIGURES = PROJECT_ROOT / 'results' / 'figures'

# Crear carpetas si no existen
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
FIGURES.mkdir(parents=True, exist_ok=True)

# Constantes del proyecto
RANDOM_SEED = 42
FIGSIZE_STANDARD = (12, 6)
FIGSIZE_WIDE = (15, 5)

print(f"✓ Paths configurados:")
print(f"  - DATA_RAW: {DATA_RAW}")
print(f"  - DATA_PROCESSED: {DATA_PROCESSED}")
print(f"  - FIGURES: {FIGURES}")

## 2. Carga de Datos M5

### Contexto del Dataset M5

El dataset M5 (Makridakis-5) de Kaggle contiene datos de ventas de Walmart con:
- **30,490 productos** jerárquicamente organizados
- **10 tiendas** en 3 estados (CA, TX, WI)
- **1,941 días** de ventas históricas
- Variables: precios, eventos, SNAP (programa asistencia alimentaria)

**Estructura:**
- Ventas en formato ancho: cada columna `d_1`, `d_2`, ..., `d_1941` es un día
- Calendar: mapea días a fechas reales, eventos, días de semana
- Prices: precios semanales por producto y tienda

In [ ]:
# Cargar calendar (ligero, carga rápida)
print("Cargando calendar.csv...")
calendar = pd.read_csv(DATA_RAW / 'calendar.csv')
print(f"✓ Calendar cargado: {calendar.shape}")
print(f"  - Rango de fechas: {calendar['date'].min()} a {calendar['date'].max()}")
print()

# Mostrar primeras filas
print("Primeras filas de calendar:")
calendar.head()

In [ ]:
# Información del calendar
print("Información de calendar:")
print("="*60)
calendar.info()
print()
print("Valores únicos por columna:")
for col in calendar.columns:
    print(f"  - {col}: {calendar[col].nunique()} valores únicos")

### Cargar Ventas (Evaluation)

**Nota:** Este archivo es grande (~60MB), la carga puede tardar 10-20 segundos.

In [ ]:
# Cargar sales (más pesado)
print("Cargando sales_train_evaluation.csv...")
print("(Puede tardar 10-20 segundos)")

sales = pd.read_csv(DATA_RAW / 'sales_train_evaluation.csv')

print(f"✓ Sales cargado: {sales.shape}")
print(f"  - Productos: {sales.shape[0]:,}")
print(f"  - Columnas (identif. + días): {sales.shape[1]}")
print(f"  - Días de ventas: {sales.shape[1] - 6}")
print(f"  - Memoria: {sales.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print()

# Mostrar estructura
print("Primeras columnas (identificadores):")
sales.iloc[:5, :10]

In [ ]:
# Información del dataset de ventas
print("Información de sales:")
print("="*60)
sales.info(verbose=False)
print()

# Columnas identificadoras
id_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
print(f"Columnas identificadoras: {id_cols}")
print(f"Columnas de ventas diarias: d_1 a d_{sales.shape[1] - len(id_cols)}")
print()

# Distribución por categorías
print("Distribución de productos:")
print(f"  - Estados: {sales['state_id'].nunique()} ({sales['state_id'].unique()})")
print(f"  - Tiendas: {sales['store_id'].nunique()}")
print(f"  - Categorías: {sales['cat_id'].nunique()} ({sales['cat_id'].unique()})")
print(f"  - Departamentos: {sales['dept_id'].nunique()}")
print(f"  - Items únicos: {sales['item_id'].nunique()}")

In [ ]:
# Cargar precios
print("Cargando sell_prices.csv...")
prices = pd.read_csv(DATA_RAW / 'sell_prices.csv')

print(f"✓ Prices cargado: {prices.shape}")
print(f"  - Registros de precios: {prices.shape[0]:,}")
print()

print("Primeras filas de prices:")
prices.head()

## 3. Análisis Exploratorio de Datos (EDA)

### 3.1 Estadísticas Descriptivas de Ventas

In [ ]:
# Extraer solo columnas de días
day_cols = [col for col in sales.columns if col.startswith('d_')]
print(f"Analizando {len(day_cols)} días de ventas...")
print()

# Estadísticas por día (promedio de ventas por producto)
daily_sales = sales[day_cols]

print("Estadísticas descriptivas de ventas diarias:")
print("="*60)
stats = daily_sales.describe().T
stats['sum'] = daily_sales.sum()
print(stats.head(20))
print("...")
print(stats.tail(20))

In [ ]:
# Análisis de ventas totales por día
total_sales_per_day = daily_sales.sum(axis=0)

print("Ventas totales agregadas:")
print("="*60)
print(f"  - Total de unidades vendidas (todos los días): {total_sales_per_day.sum():,.0f}")
print(f"  - Promedio diario: {total_sales_per_day.mean():,.0f} unidades")
print(f"  - Día con más ventas: {total_sales_per_day.idxmax()} ({total_sales_per_day.max():,.0f} unidades)")
print(f"  - Día con menos ventas: {total_sales_per_day.idxmin()} ({total_sales_per_day.min():,.0f} unidades)")
print()

# Estadísticas por producto
total_sales_per_product = daily_sales.sum(axis=1)
print("Ventas por producto:")
print("="*60)
print(f"  - Promedio de ventas por producto: {total_sales_per_product.mean():,.0f} unidades")
print(f"  - Mediana: {total_sales_per_product.median():,.0f} unidades")
print(f"  - Producto con más ventas: {total_sales_per_product.max():,.0f} unidades")
print(f"  - Producto con menos ventas: {total_sales_per_product.min():,.0f} unidades")
print(f"  - Productos con 0 ventas: {(total_sales_per_product == 0).sum()}")

### 3.2 Visualización de Series Temporales

In [ ]:
# Crear serie temporal de ventas totales diarias con fechas reales
# Mapear días a fechas usando calendar
calendar['date'] = pd.to_datetime(calendar['date'])
day_to_date = dict(zip(calendar['d'], calendar['date']))

# Convertir ventas a serie temporal
ts_sales = pd.Series(
    total_sales_per_day.values,
    index=[day_to_date[d] for d in day_cols]
)
ts_sales = ts_sales.sort_index()

print(f"Serie temporal creada: {len(ts_sales)} días")
print(f"Rango: {ts_sales.index.min()} a {ts_sales.index.max()}")

In [ ]:
# Gráfico de serie temporal completa
fig, ax = plt.subplots(figsize=FIGSIZE_WIDE)

ax.plot(ts_sales.index, ts_sales.values, linewidth=0.8, color='#2E86AB')
ax.set_title('Serie Temporal de Ventas Totales Diarias (M5 Dataset)', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Fecha', fontsize=12)
ax.set_ylabel('Unidades Vendidas', fontsize=12)
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)

# Añadir línea de promedio
ax.axhline(ts_sales.mean(), color='red', linestyle='--', 
           linewidth=1.5, alpha=0.7, label=f'Promedio: {ts_sales.mean():,.0f}')
ax.legend()

plt.tight_layout()
plt.savefig(FIGURES / '01_serie_temporal_ventas_diarias.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado en: results/figures/01_serie_temporal_ventas_diarias.png")

In [ ]:
# Análisis de tendencia y estacionalidad visual
fig, axes = plt.subplots(3, 1, figsize=(15, 10))

# 1. Serie original
axes[0].plot(ts_sales.index, ts_sales.values, linewidth=0.8, color='#2E86AB')
axes[0].set_title('Ventas Diarias Originales', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Unidades')
axes[0].grid(True, alpha=0.3)

# 2. Media móvil 7 días (semanal)
ma7 = ts_sales.rolling(window=7, center=True).mean()
axes[1].plot(ts_sales.index, ma7.values, linewidth=1.5, color='#A23B72')
axes[1].set_title('Media Móvil 7 Días (Tendencia Semanal)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Unidades')
axes[1].grid(True, alpha=0.3)

# 3. Media móvil 28 días (mensual)
ma28 = ts_sales.rolling(window=28, center=True).mean()
axes[2].plot(ts_sales.index, ma28.values, linewidth=2, color='#06A77D')
axes[2].set_title('Media Móvil 28 Días (Tendencia Mensual)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Fecha')
axes[2].set_ylabel('Unidades')
axes[2].grid(True, alpha=0.3)

for ax in axes:
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(FIGURES / '01_analisis_tendencia_estacionalidad.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado en: results/figures/01_analisis_tendencia_estacionalidad.png")

### 3.3 Análisis de Distribución de Ventas

In [ ]:
# Distribución de ventas diarias (agregadas)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma
axes[0].hist(ts_sales.values, bins=50, color='#2E86AB', alpha=0.7, edgecolor='black')
axes[0].axvline(ts_sales.mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Media: {ts_sales.mean():,.0f}')
axes[0].axvline(ts_sales.median(), color='green', linestyle='--', 
                linewidth=2, label=f'Mediana: {ts_sales.median():,.0f}')
axes[0].set_title('Distribución de Ventas Diarias', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Unidades Vendidas')
axes[0].set_ylabel('Frecuencia')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Boxplot
axes[1].boxplot(ts_sales.values, vert=True)
axes[1].set_title('Boxplot de Ventas Diarias', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Unidades Vendidas')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES / '01_distribucion_ventas.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado en: results/figures/01_distribucion_ventas.png")

In [ ]:
# Análisis de valores atípicos
Q1 = ts_sales.quantile(0.25)
Q3 = ts_sales.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_low = ts_sales[ts_sales < lower_bound]
outliers_high = ts_sales[ts_sales > upper_bound]

print("Análisis de Outliers:")
print("="*60)
print(f"  - Q1 (25%): {Q1:,.0f}")
print(f"  - Q3 (75%): {Q3:,.0f}")
print(f"  - IQR: {IQR:,.0f}")
print(f"  - Límite inferior: {lower_bound:,.0f}")
print(f"  - Límite superior: {upper_bound:,.0f}")
print()
print(f"  - Outliers bajos: {len(outliers_low)} días ({len(outliers_low)/len(ts_sales)*100:.2f}%)")
print(f"  - Outliers altos: {len(outliers_high)} días ({len(outliers_high)/len(ts_sales)*100:.2f}%)")
print()

if len(outliers_high) > 0:
    print("Top 5 días con ventas más altas (posibles urgencias):")
    top5 = ts_sales.nlargest(5)
    for date, value in top5.items():
        print(f"  - {date.strftime('%Y-%m-%d')}: {value:,.0f} unidades")

## 4. Transformación de Datos

### 4.1 Convertir de Formato Ancho a Largo

**Formato actual:** Cada producto es una fila, cada día es una columna  
**Formato objetivo:** Una fila por (producto, día) con columnas: id, fecha, ventas, precio, etc.

In [ ]:
print("Transformando datos de formato ancho a largo...")
print("(Esta operación puede tardar 30-60 segundos)")
print()

# Melt: convertir columnas d_1, d_2, ... a filas
sales_long = sales.melt(
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='sales'
)

print(f"✓ Transformación completada")
print(f"  - Forma original: {sales.shape}")
print(f"  - Forma long: {sales_long.shape}")
print(f"  - Filas creadas: {sales_long.shape[0]:,}")
print()

print("Primeras filas del formato largo:")
sales_long.head(10)

In [ ]:
# Merge con calendar para obtener fechas reales
print("Agregando información temporal desde calendar...")

sales_long = sales_long.merge(
    calendar[['d', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year']],
    on='d',
    how='left'
)

# Convertir date a datetime
sales_long['date'] = pd.to_datetime(sales_long['date'])

print(f"✓ Merge completado: {sales_long.shape}")
print()
print("Dataset con fechas:")
sales_long.head(10)

In [ ]:
# Merge con precios
print("Agregando información de precios...")

sales_long = sales_long.merge(
    prices,
    on=['store_id', 'item_id', 'wm_yr_wk'],
    how='left'
)

print(f"✓ Precios agregados: {sales_long.shape}")
print(f"  - Registros con precio: {sales_long['sell_price'].notna().sum():,}")
print(f"  - Registros sin precio: {sales_long['sell_price'].isna().sum():,}")
print()

# Calcular revenue (ventas * precio)
sales_long['revenue'] = sales_long['sales'] * sales_long['sell_price']

print("Dataset final con precios y revenue:")
sales_long.head(10)

In [ ]:
# Información del dataset transformado
print("Información del dataset transformado:")
print("="*60)
sales_long.info()
print()
print(f"Memoria utilizada: {sales_long.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

### 4.2 Agregación Semanal

**Decisión técnica:** Agregamos a nivel semanal para:
1. Reducir ruido de variabilidad diaria
2. Capturar patrones semanales más claros
3. Facilitar modelización (menos datos pero más estables)
4. Horizonte de forecasting más relevante para planificación operativa

In [ ]:
print("Agregando ventas a nivel semanal...")
print()

# Agrupar por semana (usando wm_yr_wk de Walmart)
sales_weekly = sales_long.groupby(['wm_yr_wk']).agg({
    'date': 'min',  # Primer día de la semana
    'sales': 'sum',  # Total de ventas
    'revenue': 'sum',  # Total de revenue
    'sell_price': 'mean'  # Precio promedio
}).reset_index()

# Renombrar columnas
sales_weekly.columns = ['week_id', 'week_start', 'total_sales', 'total_revenue', 'avg_price']

# Ordenar por fecha
sales_weekly = sales_weekly.sort_values('week_start').reset_index(drop=True)

# Añadir week_num como índice secuencial
sales_weekly['week_num'] = range(len(sales_weekly))

print(f"✓ Agregación semanal completada")
print(f"  - Total semanas: {len(sales_weekly)}")
print(f"  - Período: {sales_weekly['week_start'].min()} a {sales_weekly['week_start'].max()}")
print(f"  - Promedio ventas/semana: {sales_weekly['total_sales'].mean():,.0f} unidades")
print()

print("Primeras semanas:")
sales_weekly.head(10)

In [ ]:
# Estadísticas descriptivas semanales
print("Estadísticas de ventas semanales:")
print("="*60)
print(sales_weekly[['total_sales', 'total_revenue', 'avg_price']].describe())

In [ ]:
# Visualización de serie temporal semanal
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Gráfico 1: Ventas semanales
axes[0].plot(sales_weekly['week_start'], sales_weekly['total_sales'], 
             linewidth=1.5, color='#2E86AB', marker='o', markersize=3)
axes[0].axhline(sales_weekly['total_sales'].mean(), color='red', 
                linestyle='--', linewidth=1.5, alpha=0.7, 
                label=f"Promedio: {sales_weekly['total_sales'].mean():,.0f}")
axes[0].set_title('Serie Temporal de Ventas Semanales', 
                  fontsize=14, fontweight='bold', pad=20)
axes[0].set_xlabel('Fecha')
axes[0].set_ylabel('Unidades Vendidas')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Gráfico 2: Revenue semanal
axes[1].plot(sales_weekly['week_start'], sales_weekly['total_revenue'], 
             linewidth=1.5, color='#06A77D', marker='o', markersize=3)
axes[1].axhline(sales_weekly['total_revenue'].mean(), color='red', 
                linestyle='--', linewidth=1.5, alpha=0.7,
                label=f"Promedio: ${sales_weekly['total_revenue'].mean():,.0f}")
axes[1].set_title('Serie Temporal de Revenue Semanal', 
                  fontsize=14, fontweight='bold', pad=20)
axes[1].set_xlabel('Fecha')
axes[1].set_ylabel('Revenue ($)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(FIGURES / '01_serie_temporal_semanal.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado en: results/figures/01_serie_temporal_semanal.png")

## 5. Validación de Datos

### Checks de calidad de datos

In [ ]:
print("Ejecutando validaciones de calidad de datos...")
print("="*60)
print()

# 1. Missing values
print("1. MISSING VALUES:")
missing = sales_weekly.isnull().sum()
if missing.sum() == 0:
    print("   ✓ No hay valores nulos en datos semanales")
else:
    print("   ⚠ Valores nulos encontrados:")
    print(missing[missing > 0])
print()

# 2. Fechas consecutivas
print("2. CONTINUIDAD TEMPORAL:")
date_diff = sales_weekly['week_start'].diff()[1:]
expected_diff = pd.Timedelta(days=7)
gaps = date_diff[date_diff != expected_diff]
if len(gaps) == 0:
    print("   ✓ Todas las semanas son consecutivas (7 días)")
else:
    print(f"   ⚠ {len(gaps)} gaps encontrados en la serie temporal")
    print(gaps)
print()

# 3. Valores negativos
print("3. VALORES NEGATIVOS:")
negative_sales = (sales_weekly['total_sales'] < 0).sum()
if negative_sales == 0:
    print("   ✓ No hay ventas negativas")
else:
    print(f"   ⚠ {negative_sales} semanas con ventas negativas")
print()

# 4. Valores extremos
print("4. VALORES EXTREMOS:")
Q1 = sales_weekly['total_sales'].quantile(0.25)
Q3 = sales_weekly['total_sales'].quantile(0.75)
IQR = Q3 - Q1
outliers = sales_weekly[
    (sales_weekly['total_sales'] < Q1 - 1.5*IQR) | 
    (sales_weekly['total_sales'] > Q3 + 1.5*IQR)
]
print(f"   - {len(outliers)} semanas con valores atípicos ({len(outliers)/len(sales_weekly)*100:.1f}%)")
print()

# 5. Rango de valores
print("5. RANGO DE VALORES:")
print(f"   - Ventas mínimas: {sales_weekly['total_sales'].min():,.0f}")
print(f"   - Ventas máximas: {sales_weekly['total_sales'].max():,.0f}")
print(f"   - Ratio max/min: {sales_weekly['total_sales'].max() / sales_weekly['total_sales'].min():.2f}x")
print()

print("="*60)
print("✓ Validación completada")

## 6. Guardar Datos Procesados

### Outputs de esta fase:

In [ ]:
# Optimizar tipos de datos antes de guardar (reducir memoria)
print("Optimizando tipos de datos...")

# Para sales_long: convertir a tipos más eficientes
sales_long['sales'] = sales_long['sales'].astype('int16')
sales_long['wday'] = sales_long['wday'].astype('int8')
sales_long['month'] = sales_long['month'].astype('int8')
sales_long['year'] = sales_long['year'].astype('int16')

# Para sales_weekly: convertir a tipos más eficientes
sales_weekly['week_num'] = sales_weekly['week_num'].astype('int16')
sales_weekly['total_sales'] = sales_weekly['total_sales'].astype('int32')

print("✓ Tipos optimizados")

In [ ]:
# Guardar datasets procesados
print("Guardando datos procesados...")
print()

# 1. Sales en formato largo (diario) - solo una muestra si es muy grande
# Para evitar archivos enormes, guardamos solo columnas esenciales
sales_daily_essential = sales_long[[
    'd', 'date', 'item_id', 'store_id', 'sales', 'sell_price', 'revenue'
]]

print("Guardando sales_daily.csv...")
sales_daily_essential.to_csv(DATA_PROCESSED / 'sales_daily.csv', index=False)
print(f"✓ sales_daily.csv guardado ({sales_daily_essential.shape[0]:,} filas)")
print(f"  Tamaño: {(DATA_PROCESSED / 'sales_daily.csv').stat().st_size / 1024**2:.2f} MB")
print()

# 2. Sales semanales (este es el principal para forecasting)
print("Guardando sales_weekly.csv...")
sales_weekly.to_csv(DATA_PROCESSED / 'sales_weekly.csv', index=False)
print(f"✓ sales_weekly.csv guardado ({len(sales_weekly)} semanas)")
print(f"  Tamaño: {(DATA_PROCESSED / 'sales_weekly.csv').stat().st_size / 1024**2:.2f} MB")
print()

print("="*60)
print("✓ Todos los datos procesados guardados en: data/processed/")

## 7. Resumen y Conclusiones

### Hallazgos Clave de la Fase 1:

In [ ]:
# Generar resumen ejecutivo
print("""\n
╔═══════════════════════════════════════════════════════════════════╗
║                    RESUMEN EJECUTIVO - FASE 1                     ║
╚═══════════════════════════════════════════════════════════════════╝

📊 DATOS CARGADOS:
  • Dataset M5 (Walmart Sales) de Kaggle
  • 30,490 productos × 1,941 días = 59.2M registros diarios
  • Período: {} a {}
  • 10 tiendas en 3 estados (CA, TX, WI)

📈 AGREGACIÓN SEMANAL:
  • {} semanas de datos procesados
  • Promedio: {:,.0f} unidades/semana
  • Total vendido: {:,.0f} unidades
  • Revenue total: ${:,.2f}

✅ CALIDAD DE DATOS:
  • ✓ Sin valores nulos en datos semanales
  • ✓ Serie temporal continua sin gaps
  • ✓ Valores en rangos esperados
  • ✓ {} outliers identificados ({:.1f}%)

📁 OUTPUTS GENERADOS:
  • data/processed/sales_daily.csv
  • data/processed/sales_weekly.csv
  • 3 visualizaciones guardadas en results/figures/

🎯 PRÓXIMOS PASOS (Fase 2):
  1. Simulación de urgencias sintéticas
  2. Análisis exploratorio profundo (EDA)
  3. Identificación de patrones estacionales
  4. Definición de ground truth para validación

═══════════════════════════════════════════════════════════════════
✓ Fase 1 completada exitosamente
═══════════════════════════════════════════════════════════════════
""".format(
    sales_weekly['week_start'].min().strftime('%Y-%m-%d'),
    sales_weekly['week_start'].max().strftime('%Y-%m-%d'),
    len(sales_weekly),
    sales_weekly['total_sales'].mean(),
    sales_weekly['total_sales'].sum(),
    sales_weekly['total_revenue'].sum(),
    len(outliers),
    len(outliers)/len(sales_weekly)*100
))

---

## Notas Técnicas

### Decisiones Tomadas:

1. **Agregación semanal:** Se eligió agregación semanal sobre diaria para reducir ruido y capturar patrones más estables relevantes para planificación operativa.

2. **Dataset completo:** Se usó `sales_train_evaluation.csv` (1,941 días) en lugar de `sales_train_validation.csv` (1,913 días) para maximizar datos de entrenamiento.

3. **Formato largo:** Se transformó de formato ancho (días como columnas) a formato largo (días como filas) para facilitar análisis temporal y merge con otras tablas.

4. **Optimización de memoria:** Se convirtieron columnas a tipos de datos más eficientes (int16, int8) para reducir uso de memoria (~60% reducción).

5. **Outliers:** Se identificaron pero NO se eliminaron. En Fase 2 se analizarán como posibles urgencias genuinas.

### Limitaciones Conocidas:

- Algunos registros carecen de información de precios (especialmente en primeras semanas)
- Dataset M5 es de retail, no manufactura pura (pero patrones son transferibles)
- Agregación semanal puede ocultar patrones intra-semanales importantes

### Referencias:

- M5 Competition: https://www.kaggle.com/c/m5-forecasting-accuracy
- Makridakis, S., et al. (2020). "The M5 Accuracy competition"

---

**Siguiente notebook:** `02_simulacion_urgencias_eda.ipynb`